In [2]:
from akashiwo_data_scrape import *
from bs4 import BeautifulSoup
from tqdm import tqdm

In [6]:
# speacies id
spicies_ids = [45, 3]
spicies_names = ["Diaton.csv", "shtonela.csv"]

for id, names in zip(spicies_ids, spicies_names):
    pull_data(id, "2011/5/01", "2023/10/11", names)

In [96]:
colum_names = ['lat', 'lng', 'speciesValueAM', 'speciesValuePM', 
               'saisuiValueAM', 'saisuiValuePM', 'speciesNameKana', 
               'maxvalue', 'speciesId', 'icon_size']

for name in spicies_names:
    remove_colum(colum_names, name)

In [3]:
data = pd.read_csv("shtonela_info.csv")
pointIds = data["pointId"]
gatherYMDs = data["gatherYMD"]
filtered_data = remove_duplicates(pointIds, gatherYMDs)
pointIds, gatherYMDs = zip(*filtered_data)

total_rows = len(filtered_data)
split_count = 5
rows_per_split = total_rows // split_count

header = ["pointId", "gatherYMD"]

split_data = [filtered_data[i:i + rows_per_split] for i in range(0, total_rows, rows_per_split)]

for i in range(5):
    df = pd.DataFrame(split_data[i], columns=header)
    df.to_csv(f"shatonela_data_{1+i}.csv", index=False)

{'0': {0: 2497, 1: 2497, 2: 2497, 3: 2497, 4: 2497, 5: 2497, 6: 2497, 7: 2497, 8: 2498, 9: 2498, 10: 2498, 11: 2498, 12: 2498, 13: 2498, 14: 2498, 15: 2498, 16: 2499, 17: 2499, 18: 2499, 19: 2499, 20: 2499, 21: 2499, 22: 2499, 23: 10393, 24: 10395, 25: 572, 26: 572, 27: 572, 28: 572, 29: 572, 30: 572, 31: 572, 32: 572, 33: 572, 34: 572, 35: 572, 36: 572, 37: 572, 38: 572, 39: 572, 40: 572, 41: 572, 42: 572, 43: 572, 44: 572, 45: 572, 46: 572, 47: 572, 48: 572, 49: 572, 50: 572, 51: 572, 52: 572, 53: 572, 54: 572, 55: 572, 56: 572, 57: 572, 58: 572, 59: 572, 60: 572, 61: 572, 62: 572, 63: 572, 64: 572, 65: 572, 66: 572, 67: 572, 68: 572, 69: 572, 70: 572, 71: 572, 72: 572, 73: 572, 74: 572, 75: 572, 76: 572, 77: 572, 78: 572, 79: 572, 80: 572, 81: 572, 82: 572, 83: 572, 84: 572, 85: 572, 86: 572, 87: 572, 88: 572, 89: 572, 90: 572, 91: 572, 92: 572, 93: 572, 94: 572, 95: 572, 96: 572, 97: 572, 98: 572, 99: 572, 100: 572, 101: 572, 102: 572, 103: 572, 104: 572, 105: 572, 106: 572, 107: 5

In [58]:
# main_data = {}
# coordinate_data = {}
# main_df = pd.DataFrame(main_data)
# coordinate_df = pd.DataFrame(coordinate_data)
# headers_to_skip_main = ["確定値／速報値", "事業・調査名"]

def scraper_for_tables(csv):
    main_data = {}
    coordinate_data = {}
    Ids = {}
    Ids_df = {}
    main_df = pd.DataFrame(main_data)
    coordinate_df = pd.DataFrame(coordinate_data)
    Ids_temp = pd.DataFrame(Ids)
    Ids_df = pd.DataFrame(Ids_df)
    headers_to_skip_main = ["確定値／速報値", "事業・調査名"]

    data = pd.read_csv(csv)
    pointIds = data['pointId']
    gatherYMDs = data['gatherYMD']
    filtered_data = zip(pointIds, gatherYMDs)

    j = 0
    for i, (pointId, gatherYMD) in enumerate(tqdm(filtered_data, total=len(pointIds), dynamic_ncols=True)):
        html_data = requests.get(f"https://akashiwo.jp/private/akashiwoListInit.php?qpoint_id={str(pointId)}&qspecies_id=3&qgather_ymd_s=&qgather_ymd_e={str(gatherYMD)}")
        html_data.encoding = 'utf-8'
        soup = BeautifulSoup(html_data.text, 'html.parser')
        tables = soup.find_all('table')
        if len(tables) >= 2:
            times_to_duplicate, main_df = parse_main_table(tables[1], headers_to_skip_main, main_df)
            coordinate_df = parse_coordinate_table(tables[0], coordinate_df, times_to_duplicate)
            Ids = {"pointId": [pointId] * times_to_duplicate}
            Ids_temp = pd.DataFrame(Ids)
            Ids_df = pd.concat([Ids_df, Ids_temp], ignore_index=True).fillna(0)
        if j == 10:
            combined_data_noId = coordinate_df.merge(main_df, left_index=True, right_index=True)
            combined_data = Ids_df.merge(combined_data_noId, left_index=True, right_index=True)
            combined_data.to_csv("shatonela.csv", index=False)
            break
        j+=1

In [59]:
scraper_for_tables("shatonela_data_1.csv")

  0%|          | 0/51752 [00:00<?, ?it/s]

  0%|          | 19/51752 [00:01<1:17:12, 11.17it/s]
